# Rechunk the kerchunked dataset

In [1]:
import fsspec
import fsspec.implementations.reference
import zarr
import xarray as xr
from pathlib import Path

from rechunker import rechunk

In [2]:
import rechunker
rechunker.__version__

'0.5.1'

In [3]:
import zarr
zarr.__version__

'2.13.3'

#### Start a Dask Gateway cluster
Use a custom helper function `ebd.start_dask_cluster` to set options on this cluster.  We don't have to use this helper, it just cuts down on lines of code in notebooks.

In [4]:
import sys
import os
sys.path.append('/shared/users/rsignell/lib')
import ebdpy as ebd
os.environ['AWS_PROFILE'] = 'esip-qhub'  # use env vars for AWS credentials to write

client, cluster, gateway = ebd.start_dask_cluster(
    profile=os.environ['AWS_PROFILE'],
    worker_max=30,
    region='us-west-2', 
    worker_profile='Medium Worker',
    use_existing_cluster=True,
    adaptive_scaling=False, 
    wait_for_cluster=False, 
    propagate_env=True)

Region: us-west-2
Existing Dask clusters:
Cluster Index c_idx: 0 / Name: dev.cfce53af031244b2ba9880afa0165089 ClusterStatus.RUNNING
Using existing cluster [0].
Setting Fixed Scaling workers=30
Reconnect client to clear cache
client.dashboard_link (for new browser tab/window or dashboard searchbar in Jupyterhub):
https://nebari.esipfed.org/gateway/clusters/dev.cfce53af031244b2ba9880afa0165089/status
Propagating environment variables to workers
Using environment: users/users-pangeo


#### Open Kerchunked Dataset JSON for one year

#### Open Entire Kerchunked Dataset, lazy loaded from parquet

In [5]:
s3_lazy_refs = 's3://esip-qhub-public/nwm/LDAS-1k/lazyrefs'

In [45]:
%%time
fs = fsspec.implementations.reference.DFReferenceFileSystem(s3_lazy_refs, lazy=True, target_options={"anon": True},
                                                            remote_protocol="s3", remote_options={"anon": True})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", chunks={'time':1, 'y':3840, 'x':4608}, 
                     backend_kwargs=dict(consolidated=False))

CPU times: user 3.17 s, sys: 112 ms, total: 3.28 s
Wall time: 3.8 s


In [46]:
ds

<xarray.Dataset>
Dimensions:   (time: 116631, y: 3840, x: 4608, vis_nir: 2, soil_layers_stag: 4)
Coordinates:
  * time      (time) datetime64[ns] 1979-02-01T03:00:00 ... 2020-12-31T21:00:00
  * x         (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y         (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Dimensions without coordinates: vis_nir, soil_layers_stag
Data variables: (12/21)
    ACCET     (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ACSNOM    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ALBEDO    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ALBSND    (time, y, vis_nir, x) float64 dask.array<chunksize=(1, 3840, 1, 4608), meta=np.ndarray>
    ALBSNI    (time, y, vis_nir, x) float64 dask.array<chunksize=(1, 3840, 1, 4608), meta=np.ndarray>
    COSZ      (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ...        ...
    SNOWH     (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    SOIL_M    (time, y, soil_layers_stag, x) float64 dask.array<chunksize=(1, 3840, 1, 4608), meta=np.ndarray>
    SOIL_W    (time, y, soil_layers_stag, x) float64 dask.array<chunksize=(1, 3840, 1, 4608), meta=np.ndarray>
    TRAD      (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    UGDRNOFF  (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    crs       object ...
Attributes:
    Conventions:                CF-1.6
    GDAL_DataType:              Generic
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:               v5.2.0-beta2
    model_configuration:        retrospective
    model_initialization_time:  1979-02-01_00:00:00
    model_output_type:          land
    model_output_valid_time:    1979-02-01_03:00:00
    model_total_valid_times:    472
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...

#### Select high-priority vars only

In [47]:
ds = ds[['ACCET', 'SNEQV', 'FSNO', 'crs']]

In [48]:
ds

<xarray.Dataset>
Dimensions:  (time: 116631, y: 3840, x: 4608)
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01T03:00:00 ... 2020-12-31T21:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Data variables:
    ACCET    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    SNEQV    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    FSNO     (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    crs      object ...
Attributes:
    Conventions:                CF-1.6
    GDAL_DataType:              Generic
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:               v5.2.0-beta2
    model_configuration:        retrospective
    model_initialization_time:  1979-02-01_00:00:00
    model_output_type:          land
    model_output_valid_time:    1979-02-01_03:00:00
    model_total_valid_times:    472
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...

In [49]:
ds['ACCET'].isel(time=slice(0,72))

<xarray.DataArray 'ACCET' (time: 72, y: 3840, x: 4608)>
dask.array<getitem, shape=(72, 3840, 4608), dtype=float64, chunksize=(1, 3840, 4608), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01T03:00:00 ... 1979-02-10
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       Accumulated total ET
    units:           mm
    valid_range:     [-100000, 100000000]

In [11]:
ds.attrs

{'Conventions': 'CF-1.6',
 'GDAL_DataType': 'Generic',
 'TITLE': 'OUTPUT FROM WRF-Hydro v5.2.0-beta2',
 'code_version': 'v5.2.0-beta2',
 'model_configuration': 'retrospective',
 'model_initialization_time': '1979-02-01_00:00:00',
 'model_output_type': 'land',
 'model_output_valid_time': '1979-02-01_03:00:00',
 'model_total_valid_times': 472,
 'proj4': '+proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0 +lat_2=60.0 +lat_0=40.0 +lon_0=-97.0 +x_0=0 +y_0=0 +k_0=1.0 +nadgrids=@null +wktext  +no_defs'}

#### set up zarr stores for temporary and final zarr stores on S3

In [12]:
fs_write = fsspec.filesystem('s3', anon=False, skip_instance_cache=True)

In [13]:
temp_name = 'esip-qhub/testing/usgs/nwm1km.tmp'
target_name = 'esip-qhub/testing/usgs/nwm1km.zarr'

In [ ]:
fs_write.rm(temp_name, recursive=True)
fs_write.rm(target_name, recursive=True)

In [40]:
temp_store = fs_write.get_mapper(temp_name)
target_store = fs_write.get_mapper(target_name)

In [50]:
ds = ds.drop('crs')

In [ ]:
a = len(ds.time)/(144/2)

In [42]:
b = (len(ds.x) * len(ds.y))/((96*2)*(132*2))

In [43]:
a/b

4.640266927083334

In [44]:
#client.close()

In [21]:
#from dask.distributed import Client

In [22]:
#client = Client(threads_per_worker=1)

In [23]:
#client.amm.start()

#### Rechunk!

In [51]:
rechunked = rechunk(ds.isel(time=slice(0,72)), target_chunks={'y':96*2, 'x':132*2, 'time':72},
                    target_store=target_store, temp_store=temp_store, max_mem='2.8GiB')

In [52]:
%%time
rechunked.execute(retries=10)

CPU times: user 4.46 s, sys: 165 ms, total: 4.63 s
Wall time: 1min 32s


<zarr.hierarchy.Group '/'>

In [54]:
zarr.convenience.consolidate_metadata(target_store)

<zarr.hierarchy.Group '/'>

#### Explore the rechunked dataset

In [55]:
ds2 = xr.open_dataset(target_store, engine='zarr', chunks={})

In [56]:
ds2

<xarray.Dataset>
Dimensions:  (time: 72, y: 3840, x: 4608)
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01T03:00:00 ... 1979-02-10
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Data variables:
    ACCET    (time, y, x) float64 dask.array<chunksize=(72, 192, 264), meta=np.ndarray>
    FSNO     (time, y, x) float64 dask.array<chunksize=(72, 192, 264), meta=np.ndarray>
    SNEQV    (time, y, x) float64 dask.array<chunksize=(72, 192, 264), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.6
    GDAL_DataType:              Generic
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:               v5.2.0-beta2
    model_configuration:        retrospective
    model_initialization_time:  1979-02-01_00:00:00
    model_output_type:          land
    model_output_valid_time:    1979-02-01_03:00:00
    model_total_valid_times:    472
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...

In [29]:
ds2.ACCET

<xarray.DataArray 'ACCET' (time: 144, y: 3840, x: 4608)>
dask.array<open_dataset-4e2c432fb5ab0d7258d1d4529d62748cACCET, shape=(144, 3840, 4608), dtype=float64, chunksize=(72, 192, 264), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01T03:00:00 ... 1979-02-19
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       Accumulated total ET
    units:           mm
    valid_range:     [-100000, 100000000]

In [30]:
import hvplot.xarray


In [31]:
ds2.ACCET[:,2000,2000].hvplot(x='time')

:Curve   [time]   (ACCET)